In [31]:
# pip install import_ipynb
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [55]:
import import_ipynb

In [56]:
# 지난번 데이터전처리 완료한 독립변수 반환
from Entire_process_Housing_model import housing, full_pipeline

In [22]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

array([[ 0.75019159, -0.76797718,  1.05442867, ...,  0.        ,
         0.        ,  0.        ],
       [-1.20274745,  1.27527863,  0.50057941, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.63560588, -0.75391808,  0.89618602, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.29683074,  0.94254661, -0.84448308, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.56585806, -0.65550439,  0.50057941, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.76513755, -0.77266355,  0.57970073, ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()

In [15]:
# 종속변수(레이블)도 소환
from Entire_process_Housing_model import housing_labels
housing_labels

6927     143800.0
19037    117600.0
4851     150000.0
6956     172600.0
1087      78700.0
           ...   
1374     126500.0
2796      74200.0
9677     169800.0
3628     178200.0
6961     198600.0
Name: median_house_value, Length: 16512, dtype: float64

#### 세부튜닝 - 그리드 서치 cv

In [33]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators':[3,10,30], 'max_features':[2,4,6,8]}, 
              {'bootstrap':[False], 'n_estimators':[3, 10], 'max_features':[2,3,4]},
             ]

In [34]:
# 그리드 서치
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [35]:
# 최적의 그리드 서치값 도출
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [36]:
grid_search.best_estimator_ # 최적의 검출 패러미터

RandomForestRegressor(max_features=6, n_estimators=30)

In [39]:
# 평가점수 패러미터별 확인
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(f"RMSE점수 : {np.sqrt(-mean_score)}, {params}")

RMSE점수 : 62533.29447337269, {'max_features': 2, 'n_estimators': 3}
RMSE점수 : 54650.091633386575, {'max_features': 2, 'n_estimators': 10}
RMSE점수 : 52077.79591799918, {'max_features': 2, 'n_estimators': 30}
RMSE점수 : 58925.56863419812, {'max_features': 4, 'n_estimators': 3}
RMSE점수 : 52124.96196773841, {'max_features': 4, 'n_estimators': 10}
RMSE점수 : 49986.688279219976, {'max_features': 4, 'n_estimators': 30}
RMSE점수 : 58651.32021589947, {'max_features': 6, 'n_estimators': 3}
RMSE점수 : 51981.48926276242, {'max_features': 6, 'n_estimators': 10}
RMSE점수 : 49646.981059930746, {'max_features': 6, 'n_estimators': 30}
RMSE점수 : 58598.93844793059, {'max_features': 8, 'n_estimators': 3}
RMSE점수 : 52374.42908034938, {'max_features': 8, 'n_estimators': 10}
RMSE점수 : 49749.794183224105, {'max_features': 8, 'n_estimators': 30}
RMSE점수 : 61424.385269269915, {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
RMSE점수 : 53697.20999813007, {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
RMSE점수 :

In [77]:
# 서포트 벡터머신을 그리드 서치를 통해 점수 계산

# SVM 회귀 모델 사용
from sklearn import svm

grid_search_svm = GridSearchCV(estimator=svm.SVR(),
             param_grid={'C': [1, 2, 5, 7, 10], 'gamma': [0.001, 0.002, 0.01, 0.02, 0.1, 0.2, 0.5],
            'kernel': ('linear', 'rbf')}, cv = 5, 
            scoring='neg_mean_squared_error', return_train_score=True)
grid_search_svm

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 2, 5, 7, 10],
                         'gamma': [0.001, 0.002, 0.01, 0.02, 0.1, 0.2, 0.5],
                         'kernel': ('linear', 'rbf')},
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
grid_search_svm.fit(housing_prepared, housing_labels)

In [ ]:
# SVM의 최적 parameter 도출
grid_search_svm.best_params_

In [ ]:
grid_search_svm.best_estimator_ # 최적의 검출 패러미터

In [ ]:
# 평가점수 (svm) 패러미터별 확인
cvres_svm = grid_search_svm.cv_results_
for mean_score, params in zip(cvres_svm["mean_test_score"], cvres_svm["params"]):
    print(f"RMSE점수 : {np.sqrt(-mean_score)}, {params}")

#### 모델 지정, 피쳐 중요도 분석

In [44]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances
# 피쳐의 상대적인 중요도

array([7.98595010e-02, 6.97219979e-02, 4.46144076e-02, 1.96896989e-02,
       1.85302296e-02, 1.90660826e-02, 1.67202990e-02, 3.66334067e-01,
       8.19690475e-02, 1.09753520e-01, 1.64641565e-02, 1.45573003e-01,
       2.18376501e-04, 6.09583420e-03, 5.38977880e-03])

In [45]:
# 피쳐 이름 서술
from Entire_process_Housing_model import num_attribs
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs+extra_attribs+cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.3663340667752836, 'median_income'),
 (0.1455730032390434, '<1H OCEAN'),
 (0.10975351991884601, 'pop_per_hhold'),
 (0.08196904745098646, 'rooms_per_hhold'),
 (0.07985950102536842, 'longitude'),
 (0.06972199793812904, 'latitude'),
 (0.04461440755147771, 'housing_median_age'),
 (0.019689698870032746, 'total_rooms'),
 (0.01906608264175555, 'population'),
 (0.0185302295708839, 'total_bedrooms'),
 (0.016720298983827177, 'households'),
 (0.01646415652904247, 'bedrooms_per_room'),
 (0.006095834199630722, 'ISLAND'),
 (0.005389778804427372, 'NEAR BAY'),
 (0.00021837650126555177, 'INLAND')]

#### 최종모델 평가

In [46]:
from Entire_process_Housing_model import strat_test_set
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
# 테스트셋

In [48]:
from sklearn.metrics import mean_squared_error
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

48382.733803957366

In [50]:
# 신뢰구간 계산
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions-y_test) **2
np.sqrt(stats.t.interval(confidence, len(squared_errors) -1, 
                        loc=squared_erros.mean(),
                        scale=stats.sem(squared_errors)))

array([46105.67616398, 50557.33859838])